In [ ]:
import simplejson as json
import os

for key, value in json.load(open("local.settings.json"))["Values"].items():
    os.environ[key] = str(value)

In [ ]:
from datetime import datetime, timedelta
from azure.storage.blob import (
    ContainerClient,
    ContainerSasPermissions,
    generate_container_sas,
)
container = ContainerClient.from_connection_string(conn_str=os.environ["ONSPOT_CONN_STR"], container_name="general")
sas_token = generate_container_sas(
    account_name=container.account_name,
    account_key=container.credential.account_key,
    container_name=container.container_name,
    permission=ContainerSasPermissions(write=True, read=True),
    expiry=datetime.utcnow() + timedelta(days=2),
)

from libs.openapi.clients.onspot import OnSpotAPI
api = OnSpotAPI()

session_factory = api.createRequest(("/save/geoframe/all/observations", "post"))
await session_factory(
    data={
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [-123.46478462219238, 39.07166187346857],
                            [-123.4659218788147, 39.06921298141872],
                            [-123.46278905868529, 39.0687964947246],
                            [-123.4616732597351, 39.07079560844253],
                            [-123.46478462219238, 39.07166187346857],
                        ]
                    ],
                },
                "properties": {
                    "hash": False,
                    "name": "Feature Example",
                    "start": "2023-06-08T00:00:00",
                    "end": "2023-06-19T23:59:59",
                    "callback": "https://azfunc-jeami.tun.esqads.com/api/callback",
                    "outputLocation": container.url.replace("https://", "az://") + "/path/to/blobs?" + sas_token
                },
            }
        ],
    }
)